In [10]:
import numpy as np
import pandas as pd

In [11]:
### HF factors
def get_realvar(df, lookback_len, lookback_shift):
    return df['return'].shift(lookback_shift).rolling(lookback_len).var().fillna(0)

def get_realskew(df, lookback_len, lookback_shift):
    return df['return'].shift(lookback_shift).rolling(lookback_len).skew().fillna(0)

def get_realkurtosis(df, lookback_len, lookback_shift):
    return df['return'].shift(lookback_shift).rolling(lookback_len).kurt().fillna(0)

def get_realupvar(df, lookback_len, lookback_shift):
    df['return_up'] = df['return'][df['return'] > 0]
    df['return_up'] = df['return_up'].fillna(0)
    return df['return_up'].shift(lookback_shift).rolling(lookback_len).var().fillna(0)

def get_realdownvar(df, lookback_len, lookback_shift):
    df['return_down'] = df['return'][df['return'] < 0]
    df['return_down'] = df['return_down'].fillna(0)
    return df['return_down'].shift(lookback_shift).rolling(lookback_len).var().fillna(0)

def get_ratio_upvar(df, lookback_len, lookback_shift):
    return get_realupvar(df, lookback_len, lookback_shift) / get_realvar(df, lookback_len, lookback_shift)

def get_ratio_downvar(df, lookback_len, lookback_shift):
    return get_realdownvar(df, lookback_len, lookback_shift) / get_realvar(df, lookback_len, lookback_shift)

def get_trendratio(df, lookback_len, lookback_shift):
    abs_price_diff = abs(df['price'].diff()).fillna(0)
    abs_price_diff_sum = abs_price_diff.shift(lookback_shift).rolling(lookback_len).sum().fillna(0)
    trend_ratio = (df['price']-df['price'].shift(lookback_len)).shift(lookback_shift) / abs_price_diff_sum
    return trend_ratio.replace(np.inf, 0).fillna(0)

def get_windowreturn(df, lookback_len, lookback_shift):
    return np.exp((np.log(df['return']+1)).shift(lookback_shift).rolling(lookback_len).sum())-1

def get_minreturn(df, lookback_len, lookback_shift):
    return df['return'].shift(lookback_shift).rolling(lookback_len).min().fillna(0)

def calculate_mdd(series):
    max_price = np.maximum.accumulate(series)
    drawdown = (max_price - series) / max_price
    return np.max(drawdown)

def get_mdd(df, lookback_len, lookback_shift):
    return df['price'].shift(lookback_shift).rolling(lookback_len).apply(lambda x: calculate_mdd(x), raw=True)

def get_corrVP_price(df, lookback_len, lookback_shift):
    return df['price'].shift(lookback_shift).rolling(lookback_len).corr(df['volume'].shift(lookback_shift)).fillna(0)

# def get_corrVP_avg(df, lookback_len, lookback_shift):
#     return df['price'].fillna(method='ffill').shift(lookback_shift).rolling(lookback_len).corr(df['volume'].shift(lookback_shift)).fillna(0)

def get_corrVP_mid(df, lookback_len, lookback_shift):
    return df['mid'].shift(lookback_shift).rolling(lookback_len).corr(df['volume'].shift(lookback_shift)).fillna(0)

def get_corrVR(df, lookback_len, lookback_shift):
    return df['return'].shift(lookback_shift).rolling(lookback_len).corr(df['volume'].shift(lookback_shift)).fillna(0)

def get_Amihud(df, lookback_len, lookback_shift):
    abs_return = abs(df['return'].diff()).fillna(0)
    sum_abs_return = abs_return.shift(lookback_shift).rolling(lookback_len).sum()
    return (1 / (lookback_len) * sum_abs_return / df['amount'].shift(lookback_shift)).fillna(0)

def get_BAspread(df, lookback_len, lookback_shift):
    bidsum = df["b1"]*df["b1_v"]+0.8*df["b2"]*df["b2_v"]+0.6*df["b3"]*df["b3_v"]+0.4*df["b4"]*df["b4_v"]+0.2*df["b5"]*df["b5_v"]
    asksum = df["a1"]*df["a1_v"]+0.8*df["a2"]*df["a2_v"]+0.6*df["a3"]*df["a3_v"]+0.4*df["a4"]*df["a4_v"]+0.2*df["a5"]*df["a5_v"]
    df["spread"] = (bidsum - asksum) / (bidsum + asksum)
    return df["spread"].shift(lookback_shift).rolling(lookback_len).mean().fillna(0)

def delta_V_A(a1, a1_v):
    # a1 and a1_v are ndarrays
    diff = a1[-1] - a1[0]
    if diff < 0:
        return a1_v[-1]
    elif diff == 0:
        return a1_v[-1] - a1_v[0]
    else:
        return 0

def delta_V_B(b1, b1_v):
    # b1 and b1_v are ndarrays
    diff = b1[-1] - b1[0]
    if diff < 0:
        return 0
    elif diff == 0:
        return b1_v[-1] - b1_v[0]
    else:
        return b1_v[-1]
    
def get_VOI(df, lookback_len, lookback_shift):
    delta_Va = np.zeros_like(df['a1_v'])
    for i in range(1, len(df)):
        a1_slice = df['a1'].values[i-1:i+1]
        a1_v_slice = df['a1_v'].values[i-1:i+1]
        delta_Va[i] = delta_V_A(a1_slice, a1_v_slice)
    df['delta_Va'] = delta_Va

    delta_Vb = np.zeros_like(df['b1_v'])
    for i in range(1, len(df)):
        a1_slice = df['b1'].values[i-1:i+1]
        a1_v_slice = df['b1_v'].values[i-1:i+1]
        delta_Va[i] = delta_V_A(a1_slice, a1_v_slice)
    df['delta_Vb'] = delta_Vb

    df['ori_VOI'] = df['delta_Vb'] - df['delta_Va']
    df['ori_VOI'] = df['ori_VOI'].fillna(0)
    df = df.drop(columns=['delta_Va'])
    df = df.drop(columns=['delta_Vb'])

    mean = df['ori_VOI'].shift(lookback_shift).rolling(lookback_len).mean()
    std = df['ori_VOI'].shift(lookback_shift).rolling(lookback_len).std()
    return ((df['ori_VOI'] - mean)/std).replace([np.inf, -np.inf], 0).fillna(0)

def get_BAspread_1_mean(df, lookback_len, lookback_shift):
    return (df['b1'] - df['a1']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_BAspread_2_mean(df, lookback_len, lookback_shift):
    return (df['b2'] - df['a2']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_BAspread_3_mean(df, lookback_len, lookback_shift):
    return (df['b3'] - df['a3']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_BAspread_4_mean(df, lookback_len, lookback_shift):
    return (df['b4'] - df['a4']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_BAspread_5_mean(df, lookback_len, lookback_shift):
    return (df['b5'] - df['a5']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_midprice_1_mean(df, lookback_len, lookback_shift):
#     return ((df['b1']+df['a1'])/2).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_midprice_2_mean(df, lookback_len, lookback_shift):
#     return ((df['b2']+df['a2'])/2).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_midprice_3_mean(df, lookback_len, lookback_shift):
#     return ((df['b3']+df['a3'])/2).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_midprice_4_mean(df, lookback_len, lookback_shift):
#     return ((df['b4']+df['a4'])/2).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_midprice_5_mean(df, lookback_len, lookback_shift):
#     return ((df['b5']+df['a5'])/2).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_ap_diff_1_mean(df, lookback_len, lookback_shift):
#     return (df['a2'] - df['a1']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_ap_diff_2_mean(df, lookback_len, lookback_shift):
#     return (df['a3'] - df['a1']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_ap_diff_3_mean(df, lookback_len, lookback_shift):
#     return (df['a4'] - df['a1']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_ap_diff_4_mean(df, lookback_len, lookback_shift):
#     return (df['a5'] - df['a1']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_bp_diff_1_mean(df, lookback_len, lookback_shift):
#     return (df['b1'] - df['b2']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_bp_diff_2_mean(df, lookback_len, lookback_shift):
#     return (df['b1'] - df['b3']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_bp_diff_3_mean(df, lookback_len, lookback_shift):
#     return (df['b1'] - df['b4']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_bp_diff_4_mean(df, lookback_len, lookback_shift):
#     return (df['b1'] - df['b5']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_abs_ap_diff_1_mean(df, lookback_len, lookback_shift):
#     return (abs(df['a2'] - df['a1'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_abs_ap_diff_2_mean(df, lookback_len, lookback_shift):
#     return (abs(df['a3'] - df['a1'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_abs_ap_diff_3_mean(df, lookback_len, lookback_shift):
#     return (abs(df['a4'] - df['a1'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_abs_ap_diff_4_mean(df, lookback_len, lookback_shift):
#     return (abs(df['a5'] - df['a1'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_abs_bp_diff_1_mean(df, lookback_len, lookback_shift):
#     return (abs(df['b1'] - df['b2'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_abs_bp_diff_2_mean(df, lookback_len, lookback_shift):
#     return (abs(df['b1'] - df['b3'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_abs_bp_diff_3_mean(df, lookback_len, lookback_shift):
#     return (abs(df['b1'] - df['b4'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_abs_bp_diff_4_mean(df, lookback_len, lookback_shift):
#     return (abs(df['b1'] - df['b5'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_ap_sum_mean(df, lookback_len, lookback_shift):
    return (1/5 * (df['a1'] + df['a2'] + df['a3'] + df['a4'] + df['a5'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_bp_sum_mean(df, lookback_len, lookback_shift):
    return (1/5 * (df['b1'] + df['b2'] + df['b3'] + df['b4'] + df['b5'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_av_sum_mean(df, lookback_len, lookback_shift):
    return (1/5 * (df['a1_v'] + df['a2_v'] + df['a3_v'] + df['a4_v'] + df['a5_v'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_bv_sum_mean(df, lookback_len, lookback_shift):
    return (1/5 * (df['b1_v'] + df['b2_v'] + df['b3_v'] + df['b4_v'] + df['b5_v'])).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_sum_ap_diff_mean(df, lookback_len, lookback_shift):
#     return get_ap_diff_1_mean(df, lookback_len, lookback_shift) + get_ap_diff_2_mean(df, lookback_len, lookback_shift) + get_ap_diff_3_mean(df, lookback_len, lookback_shift) + get_ap_diff_4_mean(df, lookback_len, lookback_shift)
# def get_sum_bp_diff_mean(df, lookback_len, lookback_shift):
#     return get_bp_diff_1_mean(df, lookback_len, lookback_shift) + get_bp_diff_2_mean(df, lookback_len, lookback_shift) + get_bp_diff_3_mean(df, lookback_len, lookback_shift) + get_bp_diff_4_mean(df, lookback_len, lookback_shift)

# def get_deriv_ap_1_mean(df, lookback_len, lookback_shift):
#     return (df['a1'] - df['a1'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_ap_2_mean(df, lookback_len, lookback_shift):
#     return (df['a2'] - df['a2'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_ap_3_mean(df, lookback_len, lookback_shift):
#     return (df['a3'] - df['a3'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_ap_4_mean(df, lookback_len, lookback_shift):
#     return (df['a4'] - df['a4'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_ap_5_mean(df, lookback_len, lookback_shift):
#     return (df['a5'] - df['a5'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_bp_1_mean(df, lookback_len, lookback_shift):
#     return (df['b1'] - df['b1'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_bp_2_mean(df, lookback_len, lookback_shift):
#     return (df['b2'] - df['b2'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_bp_3_mean(df, lookback_len, lookback_shift):
#     return (df['b3'] - df['b3'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_bp_4_mean(df, lookback_len, lookback_shift):
#     return (df['b4'] - df['b4'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
# def get_deriv_bp_5_mean(df, lookback_len, lookback_shift):
#     return (df['b5'] - df['b5'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_av_1_mean(df, lookback_len, lookback_shift):
    return (df['a1_v'] - df['a1_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_av_2_mean(df, lookback_len, lookback_shift):
    return (df['a2_v'] - df['a2_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_av_3_mean(df, lookback_len, lookback_shift):
    return (df['a3_v'] - df['a3_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_av_4_mean(df, lookback_len, lookback_shift):
    return (df['a4_v'] - df['a4_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_av_5_mean(df, lookback_len, lookback_shift):
    return (df['a5_v'] - df['a5_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_bv_1_mean(df, lookback_len, lookback_shift):
    return (df['b1_v'] - df['b1_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_bv_2_mean(df, lookback_len, lookback_shift):
    return (df['b2_v'] - df['b2_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_bv_3_mean(df, lookback_len, lookback_shift):
    return (df['b3_v'] - df['b3_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_bv_4_mean(df, lookback_len, lookback_shift):
    return (df['b4_v'] - df['b4_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
def get_deriv_bv_5_mean(df, lookback_len, lookback_shift):
    return (df['b5_v'] - df['b5_v'].shift(2)).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)

def get_depth_price_range(df, lookback_len, lookback_shift):
    return (df['a1'].shift(lookback_shift).rolling(lookback_len).max() / df['a1'].shift(lookback_shift).rolling(lookback_len).min() - 1).fillna(0)

import numba as nb

@nb.jit(nopython=True)
def age(prices):
    last_value = prices[-1]
    age = 0
    for i in range(2, len(prices)):
        if prices[-i] != last_value:
            return age
        age += 1
    return age

def get_BAage(df, lookback_len, lookback_shift):
    return df['b1'].shift(lookback_shift).rolling(lookback_len).apply(age, engine='numba', raw=True).fillna(0)

def get_cofi(df, lookback_len, lookback_shift):
    a = df['b1_v']*np.where(df['b1'].diff()>=0, 1, 0)
    b = df['b1_v'].shift()*np.where(df['b1'].diff()<=0, 1, 0)
    c = df['a1_v']*np.where(df['a1'].diff()>=0, 1, 0)
    d = df['a1_v'].shift()*np.where(df['a1'].diff()<=0, 1, 0)
    return (a-b-c+d).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)

def get_bp_rank(df, lookback_len, lookback_shift):
    return ((df['b1'].shift(lookback_shift).rolling(lookback_len).rank()) / lookback_len*2 - 1).fillna(0)

def get_ap_rank(df, lookback_len, lookback_shift):
    return ((df['a1'].shift(lookback_shift).rolling(lookback_len).rank()) / lookback_len*2 - 1).fillna(0)

@nb.jit(nopython=True)
def first_location_of_maximum(x):
    max_value = max(x)
    for loc in range(len(x)):
        if x[loc] == max_value:
            return loc + 1
        
def get_price_idxmax(df, lookback_len, lookback_shift):
    return df['a1'].shift(lookback_shift).rolling(lookback_len).apply(first_location_of_maximum, engine='numba', raw=True).fillna(0)

@nb.jit(nopython=True)
def mean_second_derivative_centra(x):
    sum_value = 0
    for i in range(len(x)-5):
        sum_value += (x[i+5]-2*x[i+3]+x[i])/2
    return sum_value/(2*(len(x)-5))

def get_center_deri_two(df, lookback_len, lookback_shift):
    return df['a1'].shift(lookback_shift).rolling(lookback_len).apply(mean_second_derivative_centra, engine='numba', raw=True).fillna(0)

def get_quasi(df, lookback_len, lookback_shift):
    return df['a1'].diff(1).abs().shift(lookback_shift).rolling(lookback_len).sum().fillna(0)

def get_weighted_price_to_mid(df, lookback_len, lookback_shift):
    avs = df[['a1_v', 'a2_v', 'a3_v', 'a4_v', 'a5_v']].values
    bvs = df[['b1_v', 'b2_v', 'b3_v', 'b4_v', 'b5_v']].values
    aps = df[['a1', 'a2', 'a3', 'a4', 'a5']].values
    bps = df[['b1', 'b2', 'b3', 'b4', 'b5']].values
    return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)

In [12]:
lookback_len = [1, 2, 4, 8, 16, 32, 64, 128, 256]
lookback_shift = 0

for i, length in enumerate(lookback_len):
    if i == 0:
        functions = {f'{k}_{lookback_shift}_{length}': (v, length, lookback_shift) for k, v in globals().items() if callable(v) and k.startswith('get_')}
    else:
        functions.update({f'{k}_{lookback_shift}_{length}': (v, length, lookback_shift) for k, v in globals().items() if callable(v) and k.startswith('get_')})
        

print('numbers of factors:', len(functions))

numbers of factors: 414


In [13]:
def preprocess(df):
    df = df.rename(columns={'Unnamed: 0': 'tick'})
    df['lastPx'] = df['lastPx'].fillna(method='ffill')
    df['BP1'] = df['BP1'].replace(0, np.nan).fillna(method='ffill')
    df['SP1'] = df['SP1'].replace(0, np.nan).fillna(method='ffill')
    # assert df['avg_price'].isna().sum() == 0
    df = df[df['BP1'] != 0]
    df = df[df['SP1'] != 0]
    df['mid'] = (df['BP1'] + df['SP1']) / 2
    df['diff_v'] = (df['volume'] - df['volume'].shift(1)).fillna(0)
    df['return'] = (df['mid'] / df['mid'].shift(1) - 1).fillna(0)
    df = df.rename(columns={'lastPx': 'price', 'BP1':'b1', 'BP2':'b2', 'BP3':'b3', 'BP4':'b4', 'BP5':'b5', 
                            'SP1':'a1', 'SP2':'a2', 'SP3':'a3', 'SP4':'a4', 'SP5':'a5',
                            'BV1':'b1_v', 'BV2':'b2_v', 'BV3':'b3_v', 'BV4':'b4_v', 'BV5':'b5_v',
                            'SV1':'a1_v', 'SV2':'a2_v', 'SV3':'a3_v', 'SV4':'a4_v', 'SV5':'a5_v',
                            'volume': 'volume_sum', 'diff_v': 'volume'})
    df['amount'] = df['price'] * df['volume']
    return df

In [ ]:
import re

# 定义一个函数，用于检查列名是否包含不支持的特殊 JSON 字符
def check_col_name(df):
    pattern = re.compile(r'[\s#$.\[\]]')
    illegal_cols = [col for col in df.columns if pattern.search(col)]
    if len(illegal_cols) > 0:
        print('以下列名包含不支持的特殊 JSON 字符:')
        print(illegal_cols)
        # 将不支持的特殊 JSON 字符替换为下划线
        new_cols = [re.sub(pattern, '_', col) for col in df.columns]
        df.columns = new_cols
        print('列名已修改为:')
        print(df.columns)
    else:
        print('列名符合规则，无需修改')

# 示例代码
import pandas as pd

# 检查列名并修改不符合规则的列名
check_col_name(df_prc)

In [15]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import os

In [16]:
import joblib

In [ ]:
factors = os.listdir('./factors/')
for fac in tqdm(factors):
    df_prc = pd.read_csv('./factors/'+fac)
    df_prc = df_prc.iloc[:, 1:]
    # 定义滚动训练的窗口大小
    window_size = 5
    date_list = df_prc['date'].unique().tolist()

    # 定义 LASSO 模型的参数
    alpha = 0.0001
    tol = 1e-4
    max_iter = 1000
    # 定义 LASSO 模型
    model = Lasso(alpha=alpha, tol=tol, max_iter=max_iter)

    # 定义用于存储每个 tick 预测结果的列表
    r2_list = []
    ic_list = []

    # 定义用于存储特征重要性的字典
    feature_importances = {}
    importance_name = pd.DataFrame(index=date_list[window_size:], columns=range(10))
    importance_value = pd.DataFrame(index=date_list[window_size:], columns=range(10))

    # 对于每个滚动窗口
    for i in range(window_size, len(date_list)):
        # 选择训练数据和目标值
        X_train = df_prc.loc[(df_prc['date'] < date_list[i]) & (df_prc['date'] >= date_list[i-window_size]), :]
        X_train = X_train.drop(['tick', 'date', 'time'], axis=1)

        y_train = X_train['return'].shift(-1).fillna(0)
        X_train = X_train.drop(['return'], axis=1).replace([np.inf, -np.inf], 0).fillna(0)

        # 选择测试数据
        X_test = df_prc.loc[(df_prc['date'] == date_list[i]), :]
        X_test = X_test.drop(['tick', 'date', 'time'], axis=1)

        y_test = X_test['return'].shift(-1).fillna(0)
        X_test = X_test.drop(['return'], axis=1).replace([np.inf, -np.inf], 0).fillna(0)

        # 标准化训练数据和测试数据
        feature_cols = X_train.columns
        # scaler = StandardScaler()
        # X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        # X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # 训练模型
        model.fit(X_train, y_train)

        # 进行预测
        y_pred = model.predict(X_test)

        # 计算预测值和真实值之间的R方
        r2 = r2_score(y_test, y_pred)
        corr = np.corrcoef(y_test, y_pred)[0, 1]

        # 将R方添加到列表中
        r2_list.append(r2)
        ic_list.append(corr)

        # 记录特征重要性
        for feature, coef in zip(feature_cols, model.coef_):
            if feature not in feature_importances:
                feature_importances[feature] = []
            feature_importances[feature].append(coef)

        fi = pd.DataFrame(index=feature_cols, columns=['value'])
        fi['value'] = model.coef_
        fi = fi[fi.value != 0]
        fi = fi.sort_values(by='value', ascending=False)
        fi = fi.iloc[:10]
        importance_name.iloc[i-window_size, :len(fi)] = fi.index.tolist()
        importance_value.iloc[i-window_size, :len(fi)] = fi['value'].tolist()
        print(f'Total training number: {len(date_list)-window_size}, completed: {i-window_size+1}, IC: {corr}')

    # 计算所有天预测的平均 R2 值
    r2_mean = np.mean(r2_list)
    ic_mean = np.mean(ic_list)

    # 输出结果
    # print('特征重要性:', feature_importances)
    print('平均 R2 值:', r2_mean)
    print('平均 IC 值:', ic_mean)
    importance_name.to_csv('./lasso_importance_name_'+fac[8:])
    importance_value.to_csv('./lasso_importance_value_'+fac[8:])

In [17]:
filename = os.listdir('./data/2330/')
n = len(filename)
new_filename = []
for i in range(n):
    if filename[i][0:4] == '2330':
        new_filename.append(filename[i])
month = []
for file in new_filename:
    month.append(int(file[8:14]))
month.sort()

In [19]:

m = 202308
df = pd.read_csv('./data/2330/2330_md_'+str(m)+'_'+str(m)+'.csv')
date_list = df['date'].unique().tolist()

for i, date in enumerate(date_list):
    if i == 0:
        df_prc = preprocess(df[df['date'] == date])
    else:
        df_prc = pd.concat([df_prc, preprocess(df[df['date'] == date])])
for name, (func, *args) in functions.items():
    if 'get_ipython' in name:
        continue
    # print(name)
    result = func(df_prc, *args)
    var_name = name.replace('get_', '')
    df_prc = pd.concat([df_prc, result.rename(var_name)], axis=1)
#     df_prc.to_csv('./factors_2330_'+file[8:14]+'.csv')
window_size = 5
date_list = df_prc['date'].unique().tolist()

# 定义 LASSO 模型的参数
alpha = 0.0001
tol = 1e-4
max_iter = 1000
# 定义 LASSO 模型
model = Lasso(alpha=alpha, tol=tol, max_iter=max_iter)

# 定义用于存储每个 tick 预测结果的列表
r2_list = []
ic_list = []

# 定义用于存储特征重要性的字典
feature_importances = {}
pred_value = pd.DataFrame(index=df_prc.index, columns=['date', 'return'])
pred_value['date'] = df_prc['date']
#     importance_name = pd.DataFrame(index=date_list[window_size:], columns=range(10))
#     importance_value = pd.DataFrame(index=date_list[window_size:], columns=range(10))

# 对于每个滚动窗口
for i in range(window_size, len(date_list)):
    # 选择训练数据和目标值
    X_train = df_prc.loc[(df_prc['date'] < date_list[i]) & (df_prc['date'] >= date_list[i-window_size]), :]
    X_train = X_train.drop(['tick', 'date', 'time'], axis=1)

    y_train = X_train['return'].shift(-1).fillna(0)
    X_train = X_train.drop(['return'], axis=1).replace([np.inf, -np.inf], 0).fillna(0)

    # 选择测试数据
    X_test = df_prc.loc[(df_prc['date'] == date_list[i]), :]
    X_test = X_test.drop(['tick', 'date', 'time'], axis=1)

    y_test = X_test['return'].shift(-1).fillna(0)
    X_test = X_test.drop(['return'], axis=1).replace([np.inf, -np.inf], 0).fillna(0)
    for col in X_train.columns:
        if col != 'price':
            del X_train[col]
            del X_test[col]
        if col == 'price':
            break
    # 标准化训练数据和测试数据
    feature_cols = X_train.columns
    # scaler = StandardScaler()
    # X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    # X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    # 训练模型
    model.fit(X_train, y_train)

    # 进行预测
    y_pred = model.predict(X_test)
    pred_value.loc[X_test.index[0]:X_test.index[-1], 'return'] = y_pred
    # 计算预测值和真实值之间的R方
    r2 = r2_score(y_test, y_pred)
    corr = np.corrcoef(y_test, y_pred)[0, 1]

    # 将R方添加到列表中
    r2_list.append(r2)
    ic_list.append(corr)

    # 记录特征重要性
#         for feature, coef in zip(feature_cols, model.coef_):
#             if feature not in feature_importances:
#                 feature_importances[feature] = []
#             feature_importances[feature].append(coef)

#         fi = pd.DataFrame(index=feature_cols, columns=['value'])
#         fi['value'] = model.coef_
#         fi = fi[fi.value != 0]
#         fi = fi.sort_values(by='value', ascending=False)
#         fi = fi.iloc[:10]
#         importance_name.iloc[i-window_size, :len(fi)] = fi.index.tolist()
#         importance_value.iloc[i-window_size, :len(fi)] = fi['value'].tolist()

    print(f'Total training number: {len(date_list)-window_size}, completed: {i-window_size+1}, IC: {corr}')

# 计算所有天预测的平均 R2 值
r2_mean = np.mean(r2_list)
ic_mean = np.mean(ic_list)

# 输出结果
# print('特征重要性:', feature_importances)
print('平均 R2 值:', r2_mean)
print('平均 IC 值:', ic_mean)
pred_value.to_csv('./lasso_pred_2330_'+str(m)+'.csv')
joblib.dump(model, '2330_lasso.pkl')
#     importance_name.to_csv('./lasso_importance_name_'+file[0:5]+file[8:14]+'.csv')
#     importance_value.to_csv('./lasso_importance_value_'+file[0:5]+file[8:14]+'.csv')

/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * a

Total training number: 16, completed: 1, IC: -0.07455466946751241
Total training number: 16, completed: 2, IC: 0.031627987508390756
Total training number: 16, completed: 3, IC: 0.03885893340098255
Total training number: 16, completed: 4, IC: 0.02930807526607966
Total training number: 16, completed: 5, IC: 0.0321110775748027
Total training number: 16, completed: 6, IC: 0.09693453397670662
Total training number: 16, completed: 7, IC: 0.002769909022618795
Total training number: 16, completed: 8, IC: 0.004836262754646683
Total training number: 16, completed: 9, IC: 0.027901093817352528
Total training number: 16, completed: 10, IC: -0.03768130074435846
Total training number: 16, completed: 11, IC: 0.0647230206207126
Total training number: 16, completed: 12, IC: 0.05977926187058838
Total training number: 16, completed: 13, IC: 0.044169720120260214
Total training number: 16, completed: 14, IC: -0.08412887394098445
Total training number: 16, completed: 15, IC: 0.06329639034504476
Total trainin

['2330_lasso.pkl']

In [20]:
filename = os.listdir('./data/0050/')
n = len(filename)
new_filename = []
for i in range(n):
    if filename[i][0:4] == '0050':
        new_filename.append(filename[i])
month = []
for file in new_filename:
    month.append(int(file[8:14]))
month.sort()

In [21]:

m = 202308
df = pd.read_csv('./data/0050/0050_md_'+str(m)+'_'+str(m)+'.csv')
date_list = df['date'].unique().tolist()

for i, date in enumerate(date_list):
    if i == 0:
        df_prc = preprocess(df[df['date'] == date])
    else:
        df_prc = pd.concat([df_prc, preprocess(df[df['date'] == date])])
for name, (func, *args) in functions.items():
    if 'get_ipython' in name:
        continue
    # print(name)
    result = func(df_prc, *args)
    var_name = name.replace('get_', '')
    df_prc = pd.concat([df_prc, result.rename(var_name)], axis=1)
#     df_prc.to_csv('./factors_2330_'+file[8:14]+'.csv')
window_size = 5
date_list = df_prc['date'].unique().tolist()

# 定义 LASSO 模型的参数
alpha = 0.0001
tol = 1e-4
max_iter = 1000
# 定义 LASSO 模型
model = Lasso(alpha=alpha, tol=tol, max_iter=max_iter)

# 定义用于存储每个 tick 预测结果的列表
r2_list = []
ic_list = []

# 定义用于存储特征重要性的字典
feature_importances = {}
pred_value = pd.DataFrame(index=df_prc.index, columns=['date', 'return'])
pred_value['date'] = df_prc['date']
#     importance_name = pd.DataFrame(index=date_list[window_size:], columns=range(10))
#     importance_value = pd.DataFrame(index=date_list[window_size:], columns=range(10))

# 对于每个滚动窗口
for i in range(window_size, len(date_list)):
    # 选择训练数据和目标值
    X_train = df_prc.loc[(df_prc['date'] < date_list[i]) & (df_prc['date'] >= date_list[i-window_size]), :]
    X_train = X_train.drop(['tick', 'date', 'time'], axis=1)

    y_train = X_train['return'].shift(-1).fillna(0)
    X_train = X_train.drop(['return'], axis=1).replace([np.inf, -np.inf], 0).fillna(0)

    # 选择测试数据
    X_test = df_prc.loc[(df_prc['date'] == date_list[i]), :]
    X_test = X_test.drop(['tick', 'date', 'time'], axis=1)

    y_test = X_test['return'].shift(-1).fillna(0)
    X_test = X_test.drop(['return'], axis=1).replace([np.inf, -np.inf], 0).fillna(0)
    for col in X_train.columns:
        if col != 'price':
            del X_train[col]
            del X_test[col]
        if col == 'price':
            break
    # 标准化训练数据和测试数据
    feature_cols = X_train.columns
    # scaler = StandardScaler()
    # X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    # X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    # 训练模型
    model.fit(X_train, y_train)

    # 进行预测
    y_pred = model.predict(X_test)
    pred_value.loc[X_test.index[0]:X_test.index[-1], 'return'] = y_pred
    # 计算预测值和真实值之间的R方
    r2 = r2_score(y_test, y_pred)
    corr = np.corrcoef(y_test, y_pred)[0, 1]

    # 将R方添加到列表中
    r2_list.append(r2)
    ic_list.append(corr)

    # 记录特征重要性
#         for feature, coef in zip(feature_cols, model.coef_):
#             if feature not in feature_importances:
#                 feature_importances[feature] = []
#             feature_importances[feature].append(coef)

#         fi = pd.DataFrame(index=feature_cols, columns=['value'])
#         fi['value'] = model.coef_
#         fi = fi[fi.value != 0]
#         fi = fi.sort_values(by='value', ascending=False)
#         fi = fi.iloc[:10]
#         importance_name.iloc[i-window_size, :len(fi)] = fi.index.tolist()
#         importance_value.iloc[i-window_size, :len(fi)] = fi['value'].tolist()

    print(f'Total training number: {len(date_list)-window_size}, completed: {i-window_size+1}, IC: {corr}')

# 计算所有天预测的平均 R2 值
r2_mean = np.mean(r2_list)
ic_mean = np.mean(ic_list)

# 输出结果
# print('特征重要性:', feature_importances)
print('平均 R2 值:', r2_mean)
print('平均 IC 值:', ic_mean)
pred_value.to_csv('./lasso_pred_0050_'+str(m)+'.csv')
joblib.dump(model, '0050_lasso.pkl')

#     importance_name.to_csv('./lasso_importance_name_'+file[0:5]+file[8:14]+'.csv')
#     importance_value.to_csv('./lasso_importance_value_'+file[0:5]+file[8:14]+'.csv')

/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * a

Total training number: 16, completed: 1, IC: 0.11630741714894459
Total training number: 16, completed: 2, IC: 0.12363195430692862
Total training number: 16, completed: 3, IC: 0.10018354617253669
Total training number: 16, completed: 4, IC: 0.11611864698158832
Total training number: 16, completed: 5, IC: 0.08601230750235923
Total training number: 16, completed: 6, IC: 0.053144986574939705
Total training number: 16, completed: 7, IC: 0.17413123312942222
Total training number: 16, completed: 8, IC: 0.13098774597163051
Total training number: 16, completed: 9, IC: 0.14510955610942314
Total training number: 16, completed: 10, IC: 0.04991469912098832
Total training number: 16, completed: 11, IC: 0.03526526248588555
Total training number: 16, completed: 12, IC: 0.046435476778370836
Total training number: 16, completed: 13, IC: 0.13770076967614261
Total training number: 16, completed: 14, IC: -0.03453341546394138
Total training number: 16, completed: 15, IC: 0.10416698381362957
Total training n

['0050_lasso.pkl']

In [22]:
filename = os.listdir('./data/2603/')
n = len(filename)
new_filename = []
for i in range(n):
    if filename[i][0:4] == '2603':
        new_filename.append(filename[i])
month = []
for file in new_filename:
    month.append(int(file[8:14]))
month.sort()

In [23]:

m = 202308
df = pd.read_csv('./data/2603/2603_md_'+str(m)+'_'+str(m)+'.csv')
date_list = df['date'].unique().tolist()

for i, date in enumerate(date_list):
    if i == 0:
        df_prc = preprocess(df[df['date'] == date])
    else:
        df_prc = pd.concat([df_prc, preprocess(df[df['date'] == date])])
for name, (func, *args) in functions.items():
    if 'get_ipython' in name:
        continue
    # print(name)
    result = func(df_prc, *args)
    var_name = name.replace('get_', '')
    df_prc = pd.concat([df_prc, result.rename(var_name)], axis=1)
#     df_prc.to_csv('./factors_2330_'+file[8:14]+'.csv')
window_size = 5
date_list = df_prc['date'].unique().tolist()

# 定义 LASSO 模型的参数
alpha = 0.0001
tol = 1e-4
max_iter = 1000
# 定义 LASSO 模型
model = Lasso(alpha=alpha, tol=tol, max_iter=max_iter)

# 定义用于存储每个 tick 预测结果的列表
r2_list = []
ic_list = []

# 定义用于存储特征重要性的字典
feature_importances = {}
pred_value = pd.DataFrame(index=df_prc.index, columns=['date', 'return'])
pred_value['date'] = df_prc['date']
#     importance_name = pd.DataFrame(index=date_list[window_size:], columns=range(10))
#     importance_value = pd.DataFrame(index=date_list[window_size:], columns=range(10))

# 对于每个滚动窗口
for i in range(window_size, len(date_list)):
    # 选择训练数据和目标值
    X_train = df_prc.loc[(df_prc['date'] < date_list[i]) & (df_prc['date'] >= date_list[i-window_size]), :]
    X_train = X_train.drop(['tick', 'date', 'time'], axis=1)

    y_train = X_train['return'].shift(-1).fillna(0)
    X_train = X_train.drop(['return'], axis=1).replace([np.inf, -np.inf], 0).fillna(0)

    # 选择测试数据
    X_test = df_prc.loc[(df_prc['date'] == date_list[i]), :]
    X_test = X_test.drop(['tick', 'date', 'time'], axis=1)

    y_test = X_test['return'].shift(-1).fillna(0)
    X_test = X_test.drop(['return'], axis=1).replace([np.inf, -np.inf], 0).fillna(0)
    for col in X_train.columns:
        if col != 'price':
            del X_train[col]
            del X_test[col]
        if col == 'price':
            break
    # 标准化训练数据和测试数据
    feature_cols = X_train.columns
    # scaler = StandardScaler()
    # X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    # X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

    # 训练模型
    model.fit(X_train, y_train)

    # 进行预测
    y_pred = model.predict(X_test)
    pred_value.loc[X_test.index[0]:X_test.index[-1], 'return'] = y_pred
    # 计算预测值和真实值之间的R方
    r2 = r2_score(y_test, y_pred)
    corr = np.corrcoef(y_test, y_pred)[0, 1]

    # 将R方添加到列表中
    r2_list.append(r2)
    ic_list.append(corr)

    # 记录特征重要性
#         for feature, coef in zip(feature_cols, model.coef_):
#             if feature not in feature_importances:
#                 feature_importances[feature] = []
#             feature_importances[feature].append(coef)

#         fi = pd.DataFrame(index=feature_cols, columns=['value'])
#         fi['value'] = model.coef_
#         fi = fi[fi.value != 0]
#         fi = fi.sort_values(by='value', ascending=False)
#         fi = fi.iloc[:10]
#         importance_name.iloc[i-window_size, :len(fi)] = fi.index.tolist()
#         importance_value.iloc[i-window_size, :len(fi)] = fi['value'].tolist()

    print(f'Total training number: {len(date_list)-window_size}, completed: {i-window_size+1}, IC: {corr}')

# 计算所有天预测的平均 R2 值
r2_mean = np.mean(r2_list)
ic_mean = np.mean(ic_list)

# 输出结果
# print('特征重要性:', feature_importances)
print('平均 R2 值:', r2_mean)
print('平均 IC 值:', ic_mean)
pred_value.to_csv('./lasso_pred_2603_'+str(m)+'.csv')
joblib.dump(model, '2603_lasso.pkl')

#     importance_name.to_csv('./lasso_importance_name_'+file[0:5]+file[8:14]+'.csv')
#     importance_value.to_csv('./lasso_importance_value_'+file[0:5]+file[8:14]+'.csv')

/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * aps + bvs * bps).sum(axis=1) / (avs + bvs).sum(axis=1) - df['mid']).shift(lookback_shift).rolling(lookback_len).mean().fillna(0)
/var/folders/rp/8c0pvywd07x0n19dc5yqt8fw0000gn/T/ipykernel_26304/1576437422.py:279: RuntimeWarning: invalid value encountered in divide
  return ((avs * a

Total training number: 16, completed: 1, IC: 0.15745998733676814
Total training number: 16, completed: 2, IC: 0.14703912896507848
Total training number: 16, completed: 3, IC: 0.08851757215886344
Total training number: 16, completed: 4, IC: 0.15596016750516076
Total training number: 16, completed: 5, IC: 0.21285020588166467
Total training number: 16, completed: 6, IC: 0.0834052725900034
Total training number: 16, completed: 7, IC: 0.17985947580456035


/Users/zhuzheng/opt/anaconda3/envs/torch1/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.660e-06, tolerance: 1.092e-06
  model = cd_fast.enet_coordinate_descent(


Total training number: 16, completed: 8, IC: 0.04721221946248288


/Users/zhuzheng/opt/anaconda3/envs/torch1/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.115e-04, tolerance: 1.456e-06
  model = cd_fast.enet_coordinate_descent(


...

/Users/zhuzheng/opt/anaconda3/envs/torch1/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.500e-04, tolerance: 1.162e-06
  model = cd_fast.enet_coordinate_descent(


Total training number: 16, completed: 12, IC: 0.1279674348538291


/Users/zhuzheng/opt/anaconda3/envs/torch1/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.872e-04, tolerance: 8.412e-07
  model = cd_fast.enet_coordinate_descent(


Total training number: 16, completed: 13, IC: 0.15698903199674588
Total training number: 16, completed: 14, IC: 0.07644703343631336
Total training number: 16, completed: 15, IC: 0.10488741524815048
Total training number: 16, completed: 16, IC: 0.09291295248624173
平均 R2 值: -0.017604348160443428
平均 IC 值: 0.12826045414197867


['2603_lasso.pkl']

In [ ]:
featureList = []
featureImportance = []

for key in feature_importances:
    featureList.append(key)
    featureImportance.append(np.mean(feature_importances[key]))
    
## Draw Feature Importance
# Create a dataframe to store the feature importances & Sort the importacne in descending order
dfFeatImp = pd.DataFrame({'feature': featureList, 'importance': featureImportance})
dfFeatImp = dfFeatImp.sort_values('importance', ascending = False).reset_index()
    
# Normalize the feature importances
dfFeatImp['importance_normalized'] = dfFeatImp['importance'] / dfFeatImp['importance'].sum()

# Plot the feature importances in horizontal bar charts
import matplotlib.pyplot as plt
plt.figure(figsize = (20, 12))
ax = plt.subplot()
ax.barh(list(reversed(list(dfFeatImp.index[:30]))), 
    dfFeatImp['importance_normalized'].head(30), 
    align = 'center', color = 'mediumseagreen')
    
# Set the yticks and labels
ax.set_yticks(list(reversed(list(dfFeatImp.index[:30]))))
ax.set_yticklabels(dfFeatImp['feature'].head(30))
    
# Plot labeling
plt.xlabel('Normalized Importance')
plt.title('Feature Importances')
plt.show()

In [ ]:
plt.bar([i for i in range(1, len(ic_list) + 1)], ic_list)
plt.title('IC values on testing')
plt.show()

In [ ]:
plt.bar([i for i in range(1, len(r2_list) + 1)], r2_list)
plt.title('R2 values on testing')
plt.show()